In [1]:
import json
import math
import numpy as np
import string

In [2]:
with open("export.json", "r") as read_file:
    interpro_data = json.load(read_file)


In [4]:
## header creation
import pandas as pd
header = ['target_name','t-len','query_name','q-len','e-value','score','bias','#','off','ci-e-value','i-e-value','score','bias',
         'hmm_from','hmm_to','ali_from','ali_to','env_from','env_to','acc']
# open hmm results
with open('hmmsearch.hmmer_domtblout', 'r') as f2:
    data = f2.read()



# dataframe creation
lines = []
for line in data.split('\n'):
    if line != None:
        if len(line) > 1:
            if line[0] != '#':
                lines.append(line.translate(str.maketrans('', '', string.punctuation)).split()[:20])

                

data = pd.DataFrame(lines, columns = header)
target_ids = [d[2:8] for d in data['target_name']] #id of all proteins
data['target_name'] = target_ids
data.head()

,target_name,t-len,query_name,q-len,e-value,score,bias,#,off,ci-e-value,i-e-value,score,bias,hmm_from,hmm_to,ali_from,ali_to,env_from,env_to,acc
0,O75970,2070,msacutu90,91,15e15,571,237,1,13,0019,72,68,01,36,89,167,224,154,226,072
1,O75970,2070,msacutu90,91,15e15,571,237,2,13,00035,13,92,00,34,88,279,336,274,339,081
2,O75970,2070,msacutu90,91,15e15,571,237,3,13,0032,12,61,01,46,89,419,463,401,465,081
3,O75970,2070,msacutu90,91,15e15,571,237,4,13,0017,67,69,00,35,62,577,606,568,622,083
4,O75970,2070,msacutu90,91,15e15,571,237,5,13,093,36e02,14,00,32,63,726,759,719,770,082


In [26]:
def get_interpro_positions(accession, interpro_data):
    coordinates = []
    for protein in interpro_data:
        if protein['metadata']['accession'] == accession:
            for location in protein['entries'][0]['entry_protein_locations']:
                start = location['fragments'][0]['start']
                end = location['fragments'][0]['end']
                coordinates.append((start,end))
    return coordinates
        

def get_original_positions(accession, original_data):
    
    
    accession_data = original_data[original_data['target_name'] == accession]
    starts = accession_data['ali_from'].values
    ends = accession_data['ali_to'].values
    original_coordinates = [(int(starts[i]),int(ends[i])) for i in range(len(accession_data))]
    return original_coordinates
        

def compare_coordinates(coord_set1, coord_set2, length):
    set_1 = [np.arange(x[0],x[1]) for x in coord_set1]
    set_1 = set([i for c in set_1 for i in c])
    set_2 = [np.arange(x[0],x[1]) for x in coord_set2]
    set_2 = set([i for c in set_2 for i in c])
    tp = len(set_1 & set_2)
    fp = len(set_1) - tp
    fn = len(set_2) - tp
    tn = length - tp - fp - fn
    
    recall = tp/(tp+fn)
    if tp+fp != 0:
        precision = tp/(tp+fp)
    else:
        precision = None
    fp_ratio = fp/(fp+tn)
    fn_ratio = fn/(fn+tp)
    return recall, precision, fp_ratio, fn_ratio

def get_seq_length(accession, original_data):
    return int(original_data[original_data['target_name'] == accession]['t-len'].unique())

set1 = get_interpro_positions('O75970',interpro_data)
set2 = get_original_positions('O75970', data)
l = get_seq_length('O75970',data)
print(compare_coordinates(set1, set2, l))


def compute_metrics(original_data, interpro_data):
    metrics = []
    accessions = original_data['target_name'].unique()
    for accession in accessions:
        set1 = get_interpro_positions(accession,interpro_data)
        set2 = get_original_positions(accession, original_data)
        l = get_seq_length(accession,data)
        metrics.append((accession,compare_coordinates(set1, set2, l)))
    return metrics

(0.9770354906054279, 0.47368421052631576, 0.32683846637335007, 0.022964509394572025)


In [28]:
len(compute_metrics(data, interpro_data))

53

In [23]:
int(data[data['target_name'] == 'Q96QZ7']['t-len'].unique())


1491